# Client Credentials Management

In [ ]:
import csv
import os
import pandas as pd
from IPython.display import display
import requests, urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from tenable_authen import TenableAuthentication

In [ ]:
tenableAuthentication = TenableAuthentication()
HOSTNAME = tenableAuthentication.getHostname()
HEADER = tenableAuthentication.getRequestHeader()
# HOSTNAME, HEADER

# 1. Get Credentials

In [ ]:
params = {
    "id": None,
}
params

In [ ]:
getCredURL = HOSTNAME + "/rest/credential"
getCred = requests.get(getCredURL, params=params, headers=HEADER, verify=False)
print(getCred.status_code)
getCred = getCred.json()["response"]

In [ ]:
print("Usable")
display(pd.DataFrame(getCred["usable"]))

In [ ]:
# print("Manageable")
# display(pd.DataFrame(getCred["manageable"]))

# 2. Add Credentials

## A. Prepared Parameter

### "type" : "apiGateway" | "database" | "windows" | "snmp" | "ssh"

- "type" : windows

In [ ]:
def generateWindowsPayload():
    return {
        "name": name,
        "tags": "",
        "description": "",
        "type": "windows",
        "username": cyberark_username,
        "authType": "cyberark",
        "domain": domain,
        "vault_host": cyberark_vault_host,
        "vault_port": cyberark_vault_port,
        "vault_safe": cyberark_vault_safe,
        "vault_app_id": cyberark_vault_app_id,
        "vault_folder": cyberark_vault_folder,
        "vault_use_ssl": "yes",
        "vault_verify_ssl": "no",
    }

- "type" : ssh
- "privilegeEscalation" : <string> "none" | "su" | "sudo" | "su+sudo" | "dzdo" | "pbrun" | "cisco" | ".k5login",

In [ ]:
def generateSSHPayload():
    return {
        "name": name,
        "tags": "",
        "description": "",
        "type": "ssh",
        "username": cyberark_username,
        "authType": "cyberark",
        "vault_host": cyberark_vault_host,
        "vault_port": cyberark_vault_port,
        "vault_username": cyberark_vault_username,
        "vault_password": cyberark_vault_password,
        "vault_safe": cyberark_vault_safe,
        "vault_app_id": cyberark_vault_app_id,
        "vault_folder": cyberark_vault_folder,
        "vault_use_ssl": "yes",
        "vault_verify_ssl": "no",
        "vault_address": hostname,
        "privilegeEscalation": "sudo",
        "escalationUsername": "root"
    }

## B1. Add Single Credential

In [ ]:
# General
name = ""
cyberark_username = ""
cyberark_vault_host = ""
cyberark_vault_port = ""
cyberark_vault_username = ""
cyberark_vault_password = ""
cyberark_vault_safe = ""
cyberark_vault_app_id = ""
cyberark_vault_folder = ""

# Windows
domain = ""

# SSH
hostname = ""

In [ ]:
# credentialPayload = generateWindowsPayload()
credentialPayload = generateSSHPayload()

In [ ]:
addCredURL = HOSTNAME + "/rest/credential"
grtAddCred = requests.post(addCredURL, params=credentialPayload, headers=HEADER, verify=False)
print(grtAddCred.status_code, grtAddCred.json()["response"]["id"])

## B2. Add Multiple Credential

In [ ]:
finename = os.listdir("inputs/")
finename

In [ ]:
# General
name = ""
cyberark_username = ""
cyberark_vault_host = ""
cyberark_vault_port = ""
cyberark_vault_username = ""
cyberark_vault_password = ""
cyberark_vault_safe = ""
cyberark_vault_app_id = ""
cyberark_vault_folder = ""

# Windows
domain = ""

# SSH
hostname = ""

In [ ]:
input_file = "inputs/" + finename[0]

with open(input_file, mode='r', encoding='utf-8-sig') as inp:
    reader = csv.DictReader(inp)
    for row in reader:
        name = "cyberark_" + row["os"] + "_" + row["name"]
        hostname = row["host"]
        
        print("Name : " + name, end=" ")
        print(", Hostname : " + hostname)

        # credentialPayload = generateWindowsPayload()
        credentialPayload = generateSSHPayload()
        
        grtAddCred = requests.post(addCredURL, params=credentialPayload, headers=HEADER, verify=False)
        print("[" + str(grtAddCred.status_code) + "]", end=" ")
        if int(grtAddCred.status_code) == 200:
            print("Credential ID : " + grtAddCred.json()["response"]["id"])
        else:
            print(grtAddCred.text)


# 3. Get Credential by ID

In [ ]:
credentialID = "1000000"

In [ ]:
getCredByIDURL = HOSTNAME + "/rest/credential/" + str(credentialID)
getCredByID = requests.get(getCredByIDURL, headers=HEADER, verify=False)
print(getCredByID.status_code)
getCredByID = getCredByID.json()["response"]

In [ ]:
getCredByID

# 4. Delete Credential by ID

In [ ]:
credentialID4delete = "1000000"

In [ ]:
deleteCredURL = HOSTNAME + "/rest/credential/" + str(credentialID4delete)
deleteCred = requests.delete(deleteCredURL, headers=HEADER, verify=False)
print(deleteCred.status_code)
deleteCred.json()

# 5. Get Credential Tags

In [ ]:
getCredTagURL = HOSTNAME + "/rest/credential/tag"
getCredTag = requests.get(getCredTagURL, headers=HEADER, verify=False)
print(getCredTag.status_code)
getCredTag = getCredTag.json()

In [ ]:
getCredTag